# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("output_data/cities_weather.csv")
df.head()

,Unnamed: 0,City,Latitude,Longitude,Max temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bredasdorp,-34.53,20.04,48.20,93,0,4.70,ZA,1593572642
1,1,Atuona,-9.80,-139.03,79.83,74,3,21.74,PF,1593572377
2,2,Qandala,11.47,49.87,91.29,48,100,9.57,SO,1593572539
3,3,Ancud,-41.87,-73.82,50.00,100,90,12.75,CL,1593572528
4,4,Hermanus,-34.42,19.23,59.45,65,0,3.47,ZA,1593572366


In [3]:
df.count()

Unnamed: 0         573
City               573
Latitude           573
Longitude          573
Max temperature    573
Humidity           573
Cloudiness         573
Wind Speed         573
Country            569
Date               573
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

#Drop any NaN values from the dataset
clean_df = df.dropna()

In [5]:
# Store 'Lat' and 'Lng' into  locations 
locations = clean_df[["Latitude", "Longitude"]].astype(float)

#Store Humidity value 
humidity = clean_df["Humidity"].astype(float)

# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Locate the cities matching our conditions for ideal weather:
ideal_weather = clean_df.loc[(clean_df["Max temperature"]>=80) &
                             (clean_df["Max temperature"]<=87) &
                             (clean_df["Wind Speed"]<=10) &
                             (clean_df["Cloudiness"] == 0) ]
#Dropping unnecessary column                          
new_df = ideal_weather.drop(columns = ["Unnamed: 0"])
new_df

,City,Latitude,Longitude,Max temperature,Humidity,Cloudiness,Wind Speed,Country,Date
8,Butaritari,3.07,172.79,83.35,71,0,6.96,KI,1593572645
173,Bayan,46.08,127.40,80.38,50,0,5.48,CN,1593572522
262,Tessalit,20.20,1.01,80.29,50,0,7.18,ML,1593572747
294,Baharly,38.44,57.43,83.05,24,0,7.94,TM,1593572526
329,Tura,25.52,90.22,83.34,78,0,5.26,IN,1593572496
364,Tommot,58.96,126.29,84.92,31,0,6.96,RU,1593572528
390,Khorramshahr,30.44,48.17,80.60,54,0,2.24,IR,1593572802
418,Buraidah,26.33,43.98,80.60,34,0,3.36,SA,1593572813
447,Bilma,18.69,12.92,82.49,14,0,2.64,NE,1593572825
457,Bāfq,31.61,55.41,83.34,18,0,1.34,IR,1593572830


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Store Latitude, Longitude, City and Country into Hotel_df
hotel_df = new_df[["Latitude","Longitude","City","Country"]]
#Add extra column Hotel Name
hotel_df["Hotel Name"]=''
hotel_df

C:\Users\Babette\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Latitude,Longitude,City,Country,Hotel Name
8,3.07,172.79,Butaritari,KI,
173,46.08,127.40,Bayan,CN,
262,20.20,1.01,Tessalit,ML,
294,38.44,57.43,Baharly,TM,
329,25.52,90.22,Tura,IN,
364,58.96,126.29,Tommot,RU,
390,30.44,48.17,Khorramshahr,IR,
418,26.33,43.98,Buraidah,SA,
447,18.69,12.92,Bilma,NE,
457,31.61,55.41,Bāfq,IR,


In [8]:
#Set parameters to search for hotels with 5000 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "hotel"
target_radius = 5000

# set up a parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get target coordinates type from new_df
    target_coordinates = (f"{row['Latitude']}, {row['Longitude']}")
    
    # add target coordinates to params dict
    params['location'] = target_coordinates

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:     
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
               
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index, 'Hotel Name'] = "NA"

Missing field/result... skipping.


C:\Users\Babette\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [9]:
hotel_df

,Latitude,Longitude,City,Country,Hotel Name
8,3.07,172.79,Butaritari,KI,NA
173,46.08,127.40,Bayan,CN,Longfucheng Hotel
262,20.20,1.01,Tessalit,ML,NA
294,38.44,57.43,Baharly,TM,NA
329,25.52,90.22,Tura,IN,Hotel Polo Orchid
364,58.96,126.29,Tommot,RU,Hotel Shtorm
390,30.44,48.17,Khorramshahr,IR,Green Hotel
418,26.33,43.98,Buraidah,SA,Best Western Plus Buraidah
447,18.69,12.92,Bilma,NE,NA
457,31.61,55.41,Bāfq,IR,Alamdar Hotel


In [10]:
#As we can't find any hotel in a few cities we will drop the NA values
df = hotel_df.loc[hotel_df["Hotel Name"] != 'NA']
df

,Latitude,Longitude,City,Country,Hotel Name
173,46.08,127.40,Bayan,CN,Longfucheng Hotel
329,25.52,90.22,Tura,IN,Hotel Polo Orchid
364,58.96,126.29,Tommot,RU,Hotel Shtorm
390,30.44,48.17,Khorramshahr,IR,Green Hotel
418,26.33,43.98,Buraidah,SA,Best Western Plus Buraidah
457,31.61,55.41,Bāfq,IR,Alamdar Hotel


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in df.iterrows()]
locations = df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map

# Create hotel layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5,
    info_box_content=hotel_info
)

fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))